# ML for currency prediction: USD/CHF 

## Preprocessing 

### Import & load data

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm 
import random 

import torch
import torch.nn as nn

from sklearn.metrics import confusion_matrix


#%cd "/content/drive/My Drive/ml-project-2-marmlla_2/"
import Helper.Preprocessing as pp 
import Helper.Model as ml 
import Helper.Display as ds
import Helper.Pipeline as pl

df = pd.read_csv("Data/data_daily/dataset_daily.csv")
data = []
for i in df.columns[1:]:
    data.append(df[i])

df.head(2)

### Generate Dataset 

In [ ]:
result_price_df = pd.DataFrame(index=['MSE','ACC','MEAN RET %'])
result_trend_df = pd.DataFrame(index=['ACC','MEAN RET %'])
result_price_df_std = pd.DataFrame(index=['MSE','ACC','MEAN RET %'])
result_trend_df_std = pd.DataFrame(index=['ACC','MEAN RET %'])

device = 'cpu'
LOOKBACK = 6
span_trend = 1
fraction_val = 0.125
fraction_test = 0.2    

# train_y, val_y, test_y WITH TREND LABEL: 0 or 1.
train_x, val_x, test_x, train_y, val_y, test_y, _, price_tuple= pp.generate_dataset(
  data,
  lookback_=LOOKBACK,
  trend_=True,
  span_trend_=span_trend,
  span_back_trend_= 0,
  norm_=True,
  fraction_val_=fraction_val,
  fraction_test_=fraction_test)

# rain_y_reg, val_y_reg, test_y_reg WITH PRICE LABEL: USD/CHF.
_, _, _, train_y_reg, val_y_reg, test_y_reg, norm, _= pp.generate_dataset(
  data,
  lookback_=LOOKBACK,
  trend_=False,
  span_trend_=span_trend,
  norm_=True,
  fraction_val_=fraction_val,
  fraction_test_=fraction_test)

In [ ]:
train_x[np.isnan(train_x)] = 0.5 
val_x[np.isnan(val_x)] = 0.5
test_x[np.isnan(test_x)] = 0.5
train_y_reg[np.isnan(train_y_reg)] = 0.5 
val_y_reg[np.isnan(val_y_reg)] = 0.5
test_y_reg[np.isnan(test_y_reg)] = 0.5

train_y_reg[np.isnan(train_y_reg)] , val_y_reg[np.isnan(val_y_reg)], test_y_reg[np.isnan(test_y_reg)] 

In [ ]:
buff = train_y[:,0]
positive_weight = torch.tensor(
        len(buff[buff == 0.0]) / len(buff[buff == 1.0])
    ).float().to(device)
print(positive_weight)

In [ ]:
L1 = len(train_x)
L2 = len(val_x) + L1
L3 = len(test_x) + L2 

## Model

In [ ]:
bs = 128
nl = 2
n_n = 32
tax_ = 1

### Auto-Encoder

#### Apply Auto-Encoder to the dataset 

In [ ]:
model_AutoEncoder = ml.Auto_Encoder(
    input_size=train_x.shape[2],
    nb_channel_conv=8
    )
model_AutoEncoder.load_state_dict(torch.load('Helper/model/model_AutoEncoder.pth'))
model_AutoEncoder.to(device)
model_AutoEncoder

In [ ]:
model_AutoEncoder.eval()
inp1 = torch.from_numpy(np.swapaxes(train_x,1,2))
out = model_AutoEncoder(inp1.to(device).float())
train_x_smooth = np.swapaxes(out.cpu().detach().numpy(),2,1)

inp2 = torch.from_numpy(np.swapaxes(val_x,1,2))
out = model_AutoEncoder(inp2.to(device).float())
val_x_smooth = np.swapaxes(out.cpu().detach().numpy(),2,1)

inp3 = torch.from_numpy(np.swapaxes(test_x,1,2))
out = model_AutoEncoder(inp3.to(device).float())
test_x_smooth = np.swapaxes(out.cpu().detach().numpy(),2,1)

### Price prediction

In [ ]:
# Save augmented datasets with all 17 + 17 features for future use
# RUN THE TWO CELLS IN ORDER
train_x_aug = train_x.copy()
test_x_aug = test_x.copy()
val_x_aug = val_x.copy()

In [ ]:
# Slice to extract original features
train_x = train_x_aug[:,:,::2]
test_x = test_x_aug[:,:,::2]
val_x = val_x_aug[:,:,::2]

In [ ]:
print(train_x.shape, train_x_aug.shape)

#### Baseline

##### Dense

In [ ]:
res = []
for i in range(10):
    model_Dense = ml.Dense(
        num_layers = nl,
        layer_size = nn,
        input_size = train_x.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense.to(device)
    model_Dense

    lr= 0.001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_Dense,
        train_x_ = train_x.reshape(train_x.shape[0],-1),
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x.reshape(val_x.shape[0],-1),
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)

    outputs_Dense, targets, MSE = pl.evaluate(model_Dense, test_x.reshape(test_x.shape[0],-1), test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_Dense = pp.min_max_norm_inverse(outputs_Dense.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_Dense = np.mean((outputs_Dense-t)**2)
    ACC_Dense = pl.direction_accuracy(outputs_Dense, t)
    MEAN_RET_Dense = pl.evauate_strategy(t, outputs_Dense, plot=True, tax = tax_)
    
    res.append([MSE_Dense, ACC_Dense, MEAN_RET_Dense])
    
result_price_df['Dense'] = np.mean(res, axis = 0)
result_price_df_std['Dense'] = np.std(res, axis = 0)

##### LSTM

In [ ]:
res = []
for i in range(10):   
    model_LSTM = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_LSTM.to(device)
    model_LSTM

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_LSTM,
        train_x_ = train_x,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_LSTM, targets, MSE = pl.evaluate(model_LSTM, test_x, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_LSTM = pp.min_max_norm_inverse(outputs_LSTM.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_LSTM = np.mean((outputs_LSTM-t)**2)
    ACC_LSTM = pl.direction_accuracy(outputs_LSTM, t)
    MEAN_RET_LSTM = pl.evauate_strategy(t, outputs_LSTM, plot=True, tax = tax_)
    
    res.append([MSE_LSTM, ACC_LSTM, MEAN_RET_LSTM])
    
result_price_df['LSTM'] = np.mean(res, axis = 0)
result_price_df_std['LSTM'] = np.std(res, axis = 0)
print(result_price_df['LSTM'])
print(result_price_df_std['LSTM'])

##### GRU 

In [ ]:
res = []
for i in range(10):   
    model_GRU = ml.GRU_base(
            hidden_size=32,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=32,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU.to(device)
    model_GRU

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_GRU,
        train_x_ = train_x,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)

    chunksize= 1
    plt.plot(pl.smooth_loss(train_loss, chunksize = chunksize) , label="train loss")
    plt.plot(pl.smooth_loss(val_loss, chunksize = chunksize) , label="val loss")
    plt.legend()
    plt.grid()
    plt.show() 

    outputs_GRU, targets, MSE = pl.evaluate(model_GRU, test_x, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_GRU = pp.min_max_norm_inverse(outputs_GRU.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_GRU = np.mean((outputs_GRU-t)**2)
    ACC_GRU = pl.direction_accuracy(outputs_GRU, t)
    MEAN_RET_GRU = pl.evauate_strategy(t, outputs_GRU, plot=True, tax = tax_)
    
    res.append([MSE_GRU, ACC_GRU, MEAN_RET_GRU])
    
result_price_df['GRU'] = np.mean(res, axis = 0)
result_price_df_std['GRU'] = np.std(res, axis = 0)
print(result_price_df['GRU'])
print(result_price_df_std['GRU'])

#### Add additional features for patterns

In [ ]:
train_x = train_x_aug.copy()
test_x = test_x_aug.copy()
val_x = val_x_aug.copy()

In [ ]:
print(train_x.shape, val_x.shape, test_x.shape)

##### Dense

In [ ]:
res = []
for i in range(10):    
    model_Dense = ml.Dense(
        num_layers = nl,
        layer_size = nn,
        input_size = train_x.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense.to(device)
    model_Dense

    lr= 0.001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_Dense,
        train_x_ = train_x.reshape(train_x.shape[0],-1),
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x.reshape(val_x.shape[0],-1),
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)



    outputs_Dense, targets, MSE = pl.evaluate(model_Dense, test_x.reshape(test_x.shape[0],-1), test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_Dense = pp.min_max_norm_inverse(outputs_Dense.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_Dense = np.mean((outputs_Dense-t)**2)
    ACC_Dense = pl.direction_accuracy(outputs_Dense, t)
    MEAN_RET_Dense = pl.evauate_strategy(t, outputs_Dense, plot=True, tax = tax_)
    
    res.append([MSE_Dense, ACC_Dense, MEAN_RET_Dense])
    
result_price_df['Dense_aug_feat'] = np.mean(res, axis = 0)
result_price_df_std['Dense_aug_feat'] = np.std(res, axis = 0)
print(result_price_df['Dense_aug_feat'])
print(result_price_df_std['Dense_aug_feat'])

##### LSTM

In [ ]:
res = []
for i in range(10):    
    model_LSTM = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_LSTM.to(device)
    model_LSTM

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_LSTM,
        train_x_ = train_x,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_LSTM, targets, MSE = pl.evaluate(model_LSTM, test_x, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_LSTM = pp.min_max_norm_inverse(outputs_LSTM.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_LSTM = np.mean((outputs_LSTM-t)**2)
    ACC_LSTM = pl.direction_accuracy(outputs_LSTM, t)
    MEAN_RET_LSTM = pl.evauate_strategy(t, outputs_LSTM, plot=True, tax = tax_)
    
    res.append([MSE_LSTM,ACC_LSTM,MEAN_RET_LSTM])
    
result_price_df['LSTM_aug_feat_aug_feat'] = np.mean(res, axis = 0)
result_price_df_std['LSTM_aug_feat_aug_feat'] = np.std(res, axis = 0)
print(result_price_df['LSTM_aug_feat_aug_feat'])
print(result_price_df_std['LSTM_aug_feat_aug_feat'])

##### GRU 

In [ ]:
res = []
for i in range(10):
    model_GRU = ml.GRU_base(
            hidden_size=32,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=32,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU.to(device)
    model_GRU

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_GRU,
        train_x_ = train_x,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)

    outputs_GRU, targets, MSE = pl.evaluate(model_GRU, test_x, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_GRU = pp.min_max_norm_inverse(outputs_GRU.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_GRU = np.mean((outputs_GRU-t)**2)
    ACC_GRU = pl.direction_accuracy(outputs_GRU, t)
    MEAN_RET_GRU = pl.evauate_strategy(t, outputs_GRU, plot=True, tax = tax_)
    
    res.append([MSE_GRU, ACC_GRU, MEAN_RET_GRU])
    
result_price_df['GRU_aug_feat'] = np.mean(res, axis = 0)
result_price_df_std['GRU_aug_feat'] = np.std(res, axis = 0)
print(result_price_df['GRU_aug_feat'])
print(result_price_df_std['GRU_aug_feat'])

#### Add Auto-Encoder 

##### Dense + AE

In [ ]:
res = []
for i in range(10):
    model_Dense_AE = ml.Dense(
        num_layers = nl,
        layer_size = nn,
        input_size = train_x_smooth.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense_AE.to(device)
    model_Dense_AE

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_Dense_AE,
        train_x_ = train_x_smooth.reshape(train_x_smooth.shape[0],-1),
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x_smooth.reshape(val_x_smooth.shape[0],-1),
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_Dense_AE, targets, MSE = pl.evaluate(model_Dense_AE, test_x_smooth.reshape(test_x_smooth.shape[0],-1), test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_Dense_AE = pp.min_max_norm_inverse(outputs_Dense_AE.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_Dense_AE = np.mean((outputs_Dense_AE-t)**2)
    ACC_Dense_AE = pl.direction_accuracy(outputs_Dense_AE, t)
    MEAN_RET_Dense_AE = pl.evauate_strategy(t, outputs_Dense_AE, plot=True, tax = tax_)
    
    res.append([MSE_Dense_AE, ACC_Dense_AE, MEAN_RET_Dense_AE])
    
result_price_df['Dense_AE'] = np.mean(res, axis = 0)
result_price_df_std['Dense_AE'] = np.std(res, axis = 0)
print(result_price_df['Dense_AE'])
print(result_price_df_std['Dense_AE'])


##### LSTM + AE

In [ ]:
res = []
for i in range(10):
    model_LSTM_AE = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x_smooth.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.1,
            device=device)
    model_LSTM_AE.to(device)
    model_LSTM_AE

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_LSTM_AE,
        train_x_ = train_x_smooth,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x_smooth,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_LSTM_AE, targets, MSE = pl.evaluate(model_LSTM_AE, test_x_smooth, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_LSTM_AE = pp.min_max_norm_inverse(outputs_LSTM_AE.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_LSTM_AE = np.mean((outputs_LSTM_AE-t)**2)
    ACC_LSTM_AE = pl.direction_accuracy(outputs_LSTM_AE, t)
    MEAN_RET_LSTM_AE = pl.evauate_strategy(t, outputs_LSTM_AE, plot=True, tax= tax_)

    
    res.append([MSE_LSTM_AE, ACC_LSTM_AE, MEAN_RET_LSTM_AE])
    
result_price_df['LSTM_AE'] = np.mean(res, axis = 0)
result_price_df_std['LSTM_AE'] = np.std(res, axis = 0)
print(result_price_df['LSTM_AE'])
print(result_price_df_std['LSTM_AE'])


##### GRU + AE

In [ ]:
res = []
for i in range(10):
    model_GRU_AE = ml.GRU_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x_smooth.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU_AE.to(device)
    model_GRU_AE

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_GRU_AE,
        train_x_ = train_x_smooth,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x_smooth,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_GRU_AE, targets, MSE = pl.evaluate(model_GRU_AE, test_x_smooth, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_GRU_AE = pp.min_max_norm_inverse(outputs_GRU_AE.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_GRU_AE = np.mean((outputs_GRU_AE-t)**2)
    ACC_GRU_AE = pl.direction_accuracy(outputs_GRU_AE, t)
    MEAN_RET_GRU_AE = pl.evauate_strategy(t, outputs_GRU_AE, plot=True, tax=tax_)

    res.append([MSE_GRU_AE, ACC_GRU_AE, MEAN_RET_GRU_AE])
    
result_price_df['GRU_AE'] = np.mean(res, axis = 0)
result_price_df_std['GRU_AE'] = np.std(res, axis = 0)
print(result_price_df['GRU_AE'])
print(result_price_df_std['GRU_AE'])

#### Add Regularized Loss: $L = \sum (y- \hat{y})^2 + \lambda \sum ((y- y_{prev})(y- \hat{y}))^2 $

##### Dense + AE + REG

In [ ]:
res = []
for i in range(10):
    model_Dense_AE_REG = ml.Dense(
        num_layers = nl,
        layer_size = nn,
        input_size = train_x_smooth.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense_AE_REG.to(device)
    model_Dense_AE_REG

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train_regularized(
        model=model_Dense_AE_REG,
        train_x_ = train_x_smooth.reshape(train_x_smooth.shape[0],-1),
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x_smooth.reshape(val_x_smooth.shape[0],-1),
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lambda_=0.01,
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_Dense_AE_REG, targets, MSE = pl.evaluate(model_Dense_AE_REG, test_x_smooth.reshape(test_x_smooth.shape[0],-1), test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_Dense_AE_REG = pp.min_max_norm_inverse(outputs_Dense_AE_REG.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_Dense_AE_REG = np.mean((outputs_Dense_AE_REG-t)**2)
    ACC_Dense_AE_REG = pl.direction_accuracy(outputs_Dense_AE_REG, t)
    MEAN_RET_Dense_AE_REG = pl.evauate_strategy(t, outputs_Dense_AE_REG,plot=True, tax = tax_)
    
    res.append([MSE_Dense_AE_REG, ACC_Dense_AE_REG, MEAN_RET_Dense_AE_REG])
    
result_price_df['Dense_AE_REG'] = np.mean(res, axis = 0)
result_price_df_std['Dense_AE_REG'] = np.std(res, axis = 0)
print(result_price_df['Dense_AE_REG'])
print(result_price_df_std['Dense_AE_REG'])


##### LSTM + AE + REG

In [ ]:
res = []
for i in range(10):
    model_LSTM_AE_REG = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x_smooth.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_LSTM_AE_REG.to(device)
    model_LSTM_AE_REG

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train_regularized(
        model=model_LSTM_AE_REG,
        train_x_ = train_x_smooth,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x_smooth,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        lambda_=0.01,
        device_=device,
        verbose = 1)


    outputs_LSTM_AE_REG, targets, MSE = pl.evaluate(model_LSTM_AE_REG, test_x_smooth, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_LSTM_AE_REG = pp.min_max_norm_inverse(outputs_LSTM_AE_REG.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_LSTM_AE_REG = np.mean((outputs_LSTM_AE_REG-t)**2)
    ACC_LSTM_AE_REG = pl.direction_accuracy(outputs_LSTM_AE_REG, t)
    MEAN_RET_LSTM_AE_REG = pl.evauate_strategy(t, outputs_LSTM_AE_REG, plot=True, tax=tax_)

    res.append([MSE_LSTM_AE_REG, ACC_LSTM_AE_REG, MEAN_RET_LSTM_AE_REG])
    
result_price_df['LSTM_AE_REG'] = np.mean(res, axis = 0)
result_price_df_std['LSTM_AE_REG'] = np.std(res, axis = 0)
print(result_price_df['LSTM_AE_REG'])
print(result_price_df_std['LSTM_AE_REG'])


##### GRU + AE + REG

In [ ]:
res = []
for i in range(10):
    model_GRU_AE_REG = ml.GRU_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x_smooth.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU_AE_REG.to(device)
    model_GRU_AE_REG

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train_regularized(
        model=model_GRU_AE_REG,
        train_x_ = train_x_smooth,
        train_y_ =  train_y_reg[:,0].reshape(-1,1),
        val_x_ = val_x_smooth,
        val_y_=  val_y_reg[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        lambda_=0.01,
        device_=device,
        verbose = 1)

    outputs_GRU_AE_REG, targets, MSE = pl.evaluate(model_GRU_AE_REG, test_x_smooth, test_y_reg[:,0].reshape(-1,1), device=device) 

    tuple_min_max= norm[2]
    outputs_GRU_AE_REG = pp.min_max_norm_inverse(outputs_GRU_AE_REG.reshape(-1,), tuple_min_max_=tuple_min_max) 
    t = pp.min_max_norm_inverse(targets.reshape(-1,), tuple_min_max_=tuple_min_max)

    MSE_GRU_AE_REG = np.mean((outputs_GRU_AE_REG-t)**2)
    ACC_GRU_AE_REG = pl.direction_accuracy(outputs_GRU_AE_REG, t)
    MEAN_RET_GRU_AE_REG = pl.evauate_strategy(t, outputs_GRU_AE_REG, plot=True, tax = tax_)
    
    
    res.append([MSE_GRU_AE_REG, ACC_GRU_AE_REG, MEAN_RET_GRU_AE_REG])
    
result_price_df['GRU_AE_REG'] = np.mean(res, axis = 0)
result_price_df_std['GRU_AE_REG'] = np.std(res, axis = 0)
print(result_price_df['GRU_AE_REG'])
print(result_price_df_std['GRU_AE_REG'])


### Trend prediction

#### Baseline

In [ ]:
# Slice to extract original features
train_x = train_x_aug.copy()
test_x = test_x_aug.copy()
val_x = val_x_aug.copy()

In [ ]:
print(train_x.shape, train_x_aug.shape)

In [ ]:
n_n = 32
import torch
import torch.nn as nn

##### Dense 

In [ ]:
res = []
for i in range(10):
    model_Dense_Trend = ml.Dense(
        num_layers = nl,
        layer_size = n_n,
        input_size = train_x.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense_Trend.to(device)
    model_Dense_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_Dense_Trend,
        train_x_ = train_x.reshape(train_x.shape[0],-1),
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x.reshape(val_x.shape[0],-1),
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        device_=device,
        verbose = 1)

    outputs_Dense_Trend, targets, ACC_Dense_Trend = pl.evaluate_trend(model_Dense_Trend, test_x.reshape(test_x.shape[0],-1), test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_Dense_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0], tuple_min_max_=tuple_min_max)

    MEAN_RET_Dense_Trend = pl.evauate_strategy_trend(t, outputs_Dense_Trend,plot=True, tax =tax_)

    
    res.append([ACC_Dense_Trend, MEAN_RET_Dense_Trend])
    
result_trend_df['Dense'] = np.mean(res, axis = 0)
result_trend_df_std['Dense'] = np.std(res, axis = 0)
print(result_trend_df['Dense'])
print(result_trend_df_std['Dense'])

##### LSTM

In [ ]:
res = []
for i in range(10):
    model_LSTM_Trend = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_LSTM_Trend.to(device)
    model_LSTM_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_LSTM_Trend,
        train_x_ = train_x,
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_LSTM_Trend, targets, ACC_LSTM_Trend = pl.evaluate_trend(model_LSTM_Trend, test_x, test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_LSTM_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0].reshape(-1,), tuple_min_max_=tuple_min_max)

    MEAN_RET_LSTM_Trend = pl.evauate_strategy_trend(t, outputs_LSTM_Trend,plot=True, tax=tax_)
    
    res.append([ACC_LSTM_Trend, MEAN_RET_LSTM_Trend])
    
result_trend_df['LSTM'] = np.mean(res, axis = 0)
result_trend_df_std['LSTM'] = np.std(res, axis = 0)
print(result_trend_df['LSTM'])
print(result_trend_df_std['LSTM'])

##### GRU

In [ ]:
res = []
for i in range(10):
    model_GRU_Trend = ml.GRU_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU_Trend.to(device)
    model_GRU_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_GRU_Trend,
        train_x_ = train_x,
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_GRU_Trend, targets, ACC_GRU_Trend = pl.evaluate_trend(model_GRU_Trend, test_x, test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_GRU_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0].reshape(-1,), tuple_min_max_=tuple_min_max)

    MEAN_RET_GRU_Trend = pl.evauate_strategy_trend(t, outputs_GRU_Trend, plot=True, tax=tax_)
    
    res.append([ACC_GRU_Trend, MEAN_RET_GRU_Trend])
    
result_trend_df['GRU'] = np.mean(res, axis = 0)
result_trend_df_std['GRU'] = np.std(res, axis = 0)
print(result_trend_df['GRU'])
print(result_trend_df_std['GRU'])

#### Add augmented features

In [ ]:
train_x = train_x_aug.copy()
test_x = test_x_aug.copy()
val_x = val_x_aug.copy()

In [ ]:
print(train_x.shape, val_x.shape, test_x.shape)

##### Dense

In [ ]:
res = []
for i in range(10):
    model_Dense_Trend = ml.Dense(
        num_layers = nl,
        layer_size = n_n,
        input_size = train_x.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense_Trend.to(device)
    model_Dense_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_Dense_Trend,
        train_x_ = train_x.reshape(train_x.shape[0],-1),
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x.reshape(val_x.shape[0],-1),
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        device_=device,
        verbose = 1)


    outputs_Dense_Trend, targets, ACC_Dense_Trend = pl.evaluate_trend(model_Dense_Trend, test_x.reshape(test_x.shape[0],-1), test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_Dense_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0], tuple_min_max_=tuple_min_max)

    MEAN_RET_Dense_Trend = pl.evauate_strategy_trend(t, outputs_Dense_Trend,plot=True, tax=tax_)
    
    res.append([ACC_Dense_Trend, MEAN_RET_Dense_Trend])
    
result_trend_df['Dense_aug_feat'] = np.mean(res, axis = 0)
result_trend_df_std['Dense_aug_feat'] = np.std(res, axis = 0)
print(result_trend_df['Dense_aug_feat'])
print(result_trend_df_std['Dense_aug_feat'])

##### LSTM

In [ ]:
res = []
for i in range(10):
    model_LSTM_Trend = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_LSTM_Trend.to(device)
    model_LSTM_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_LSTM_Trend,
        train_x_ = train_x,
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_LSTM_Trend, targets, ACC_LSTM_Trend = pl.evaluate_trend(model_LSTM_Trend, test_x, test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_LSTM_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0].reshape(-1,), tuple_min_max_=tuple_min_max)

    MEAN_RET_LSTM_Trend = pl.evauate_strategy_trend(t, outputs_LSTM_Trend,plot=True, tax=tax_)

    res.append([ACC_LSTM_Trend, MEAN_RET_LSTM_Trend])
    
result_trend_df['LSTM_aug_feat'] = np.mean(res, axis = 0)
result_trend_df_std['LSTM_aug_feat'] = np.std(res, axis = 0)
print(result_trend_df['LSTM_aug_feat'])
print(result_trend_df_std['LSTM_aug_feat'])


##### GRU

In [ ]:
res = []
for i in range(10):
    model_GRU_Trend = ml.GRU_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU_Trend.to(device)
    model_GRU_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_GRU_Trend,
        train_x_ = train_x,
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x,
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        lr_=lr,
        device_=device,
        verbose = 1)

    outputs_GRU_Trend, targets, ACC_GRU_Trend = pl.evaluate_trend(model_GRU_Trend, test_x, test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_GRU_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0].reshape(-1,), tuple_min_max_=tuple_min_max)

    MEAN_RET_GRU_Trend = pl.evauate_strategy_trend(t, outputs_GRU_Trend, plot=True, tax=tax_)

    res.append([ACC_GRU_Trend, MEAN_RET_GRU_Trend])
    
result_trend_df['GRU_aug_feat'] = np.mean(res, axis = 0)
result_trend_df_std['GRU_aug_feat'] = np.std(res, axis = 0)
print(result_trend_df['GRU_aug_feat'])
print(result_trend_df_std['GRU_aug_feat'])


#### Add Auto-Encoder 

##### Dense + AE

In [ ]:
res = []
for i in range(10):
    model_Dense_AE_Trend = ml.Dense(
        num_layers = nl,
        layer_size = n_n,
        input_size = train_x_smooth.reshape(train_x.shape[0],-1).shape[1],
        output_size = 1,
        dropout=0.05)
    model_Dense_AE_Trend.to(device)
    model_Dense_AE_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_Dense_AE_Trend,
        train_x_ = train_x_smooth.reshape(train_x.shape[0],-1),
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x_smooth.reshape(val_x.shape[0],-1),
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        lr_=lr,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        device_=device,
        verbose = 1)


    outputs_Dense_AE_Trend, targets, ACC_Dense_AE_Trend = pl.evaluate_trend(model_Dense_AE_Trend, test_x_smooth.reshape(test_x.shape[0],-1), test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_Dense_AE_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0], tuple_min_max_=tuple_min_max)

    MEAN_RET_Dense_AE_Trend = pl.evauate_strategy_trend(t, outputs_Dense_AE_Trend,plot=True, tax=tax_)

    res.append([ACC_Dense_AE_Trend, MEAN_RET_Dense_AE_Trend])
    
result_trend_df['Dense_AE'] = np.mean(res, axis = 0)
result_trend_df_std['Dense_AE'] = np.std(res, axis = 0)
print(result_trend_df['Dense_AE'])
print(result_trend_df_std['Dense_AE'])


##### LSTM + AE

In [ ]:
res = []
for i in range(10):
    model_LSTM_AE_Trend = ml.LSTM_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x_smooth.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_LSTM_AE_Trend.to(device)
    model_LSTM_AE_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_LSTM_AE_Trend,
        train_x_ = train_x_smooth,
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x_smooth,
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        lr_=lr,
        device_=device,
        verbose = 1)

    outputs_LSTM_AE_Trend, targets, ACC_LSTM_AE_Trend = pl.evaluate_trend(model_LSTM_AE_Trend, test_x_smooth, test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_LSTM_AE_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0].reshape(-1,), tuple_min_max_=tuple_min_max)

    MEAN_RET_LSTM_AE_Trend = pl.evauate_strategy_trend(t, outputs_LSTM_AE_Trend,plot=True, tax=tax_)
    
    res.append([ACC_LSTM_AE_Trend, MEAN_RET_LSTM_AE_Trend])
    
result_trend_df['LSTM_AE'] = np.mean(res, axis = 0)
result_trend_df_std['LSTM_AE'] = np.std(res, axis = 0)
print(result_trend_df['LSTM_AE'])
print(result_trend_df_std['LSTM_AE'])

##### GRU + AE

In [ ]:
res = []
for i in range(10):
    model_GRU_AE_Trend = ml.GRU_base(
            hidden_size=16,
            num_layers=2,
            input_size= train_x_smooth.shape[2],
            out_features_lin=16,
            out_features_end=1,
            dropout=0.05,
            device=device)
    model_GRU_AE_Trend.to(device)
    model_GRU_AE_Trend

    lr= 0.0001
    num_epochs= 1000

    train_loss, val_loss = pl.train(
        model=model_GRU_AE_Trend,
        train_x_ = train_x_smooth,
        train_y_ =  train_y[:,0].reshape(-1,1),
        val_x_ = val_x_smooth,
        val_y_=  val_y[:,0].reshape(-1, 1),
        batch_size_=bs,
        num_epochs_=num_epochs,
        criterion_ = nn.BCEWithLogitsLoss(pos_weight=positive_weight),
        lr_=lr,
        device_=device,
        verbose = 1)


    outputs_GRU_AE_Trend, targets, ACC_GRU_AE_Trend = pl.evaluate_trend(model_GRU_AE_Trend, test_x_smooth, test_y[:,0].reshape(-1,1), device=device) 

    sns.heatmap(data=confusion_matrix(targets, outputs_GRU_AE_Trend), annot=True)
    plt.plot()

    tuple_min_max= norm[2]
    t = pp.min_max_norm_inverse(test_y_reg[:,0].reshape(-1,), tuple_min_max_=tuple_min_max)

    MEAN_RET_GRU_AE_Trend = pl.evauate_strategy_trend(t, outputs_GRU_AE_Trend, plot=True, tax=tax_)
    
    res.append([ACC_GRU_AE_Trend, MEAN_RET_GRU_AE_Trend])
    
result_trend_df['GRU_AE'] = np.mean(res, axis = 0)
result_trend_df_std['GRU_AE'] = np.std(res, axis = 0)
print(result_trend_df['GRU_AE'])
print(result_trend_df_std['GRU_AE'])

## Result 

### Price

In [ ]:
result_price_df.head(3)

In [ ]:
result_price_df_std.head(3)

### Trend 

In [ ]:
result_trend_df.head(3)

In [ ]:
result_trend_df_std.head(3)